# Understanding the in-situ Solar Wind Properties with Machine Learning and Artificial Intelligence


Project Abstract: 
With the rapid increase of the in-situ measurements of the solar wind plasma, traditional data analysis methods are no longer sufficient for Heliophysics scientists to fully comprehend the scientific insights embedded within the data. Applications of Machine learning (ML) and Artificial Intelligence (AI) techniques on the solar data in order to perform feature selection, dimension reduction and clustering is the major goal of this proposal.

Project Outcome:
1). The importance ranking of the solar wind input data.
2). Visualization of the solar wind data in 2D, after the dimension reduction.
3). Labels of the clustering.
4). Prediction of the monthly sunspot number and the indexes of the HCS


# Predictive Analytics / Modeling

In [1]:
import pandas as pd
import re
import numpy as np
# from pydantic_settings import BaseSettings #not sure if it's needed
# from ydata_profiling import ProfileReport
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

#pip install ydata-profiling
#pip install pydantic-settings 


In [2]:
# Maybe delete
# import sys
# sys.path.append('/path/to/your/module')
#from fractional_date_converter import fractional_date_to_actual_date

In [3]:
#Read CSVs

#Just HCS
hcs_df = pd.read_csv('../data/HCS Cleaned.csv')
hcs_df['actual_date'] = pd.to_datetime(hcs_df['actual_date'])  

#HCS + Sunspots
hcs_sun = pd.read_csv("../data/HCS + Sunspots.csv")
hcs_sun['date'] = pd.to_datetime(hcs_sun['date']) 

#HCS + Sinsports + ACE
hcs_sun_ace= pd.read_csv("../data/HCS + Sunspots + ACE.csv")
hcs_sun_ace['date'] = pd.to_datetime(hcs_sun_ace['date']) 

In [4]:
hcs_sun_ace.head()

,year,month,SD_70,SL_70,SL_70_log10,fyear_CS,monthly_sunspots,std,observations,date,smoothed_sunspots,SD_70*10,proton_density,proton_temp,He4toprotons,proton_speed,PC1,PC2
0,1998,2,12.301438,82.789337,1.917974,1998.123,50.2,4.6,559,1998-02-01,61.745833,123.014380,10.668186,55818.621443,0.022320,377.004722,-1.249865,-0.740865
1,1998,3,12.602894,159.079344,2.008504,1998.204,82.0,5.9,571,1998-03-01,65.370833,126.028935,9.296941,71053.688126,0.030932,402.813548,-0.854332,-0.273900
2,1998,4,15.813673,380.628082,2.580501,1998.288,70.6,7.2,537,1998-04-01,69.179167,158.136730,8.991568,71262.155483,0.037028,379.456547,-1.017707,-0.005488
3,1998,5,12.578657,128.306992,2.108250,1998.371,74.0,4.8,620,1998-05-01,72.120833,125.786570,6.943908,100056.671947,0.031098,452.484785,-0.047913,-0.120904
4,1998,6,16.536440,138.153671,2.140362,1998.455,90.5,6.0,521,1998-06-01,77.295833,165.364400,8.368127,69987.706731,0.024842,414.453910,-0.684110,-0.465511


### Getting really bad results with this df. It's reduced partly because the ACE data only starts from 1998


In [5]:
#Creating copy of df and droping NAs
hcs_sun_ace_model = hcs_sun_ace.copy().dropna()

#Doing a sequential split
split_point = int(len(hcs_sun_ace_model) * 0.8)  # 80% for training, 20% for testing

#Selecting filters
features = hcs_sun_ace_model[['monthly_sunspots','proton_density', 'proton_temp', 'He4toprotons','proton_speed','year','month', 'PC1','PC2']]
features = hcs_sun_ace_model[['monthly_sunspots','year','month', 'PC1','PC2']]

#Scaling features
scaler = StandardScaler()

# Fit the scaler to your features and transform them
scaled = scaler.fit_transform(features)

#Putting them in a df for the patition
X = pd.DataFrame(scaled, columns=features.columns, index=features.index)

#Outcome variables
y_std = hcs_sun_ace_model['SD_70']
y_slope = hcs_sun_ace_model['SL_70_log10']

In [6]:
# # Training and testing split for standard deviation prediction
X_train_std = X.iloc[:split_point]
X_test_std = X.iloc[split_point:]
y_train_std = y_std.iloc[:split_point]
y_test_std = y_std.iloc[split_point:]

# # Training and testing split for slope prediction
X_train_slope = X.iloc[:split_point]
X_test_slope = X.iloc[split_point:]
y_train_slope = y_slope.iloc[:split_point]
y_test_slope = y_slope.iloc[split_point:]

In [7]:
# # Model for predicting standard deviation
model_std = LinearRegression().fit(X_train_std, y_train_std)

# # Model for predicting slope
model_slope = LinearRegression().fit(X_train_slope, y_train_slope)

# # Evaluating the models using R^2 score or other relevant metrics
std_score = model_std.score(X_test_std, y_test_std)


slope_score = model_slope.score(X_test_slope, y_test_slope)

print("STD Model R^2 with ACE:", std_score)
print("Slope Model R^2 with ACE:", slope_score)

STD Model R^2 with ACE: -7.1497496274517545
Slope Model R^2 with ACE: -8.776024666877731


### Trying modeling without the ACE data features

In [8]:
#Creating copy of df and droping NAs
hcs_sun_model = hcs_sun.copy().dropna()

#Doing a sequential split
split_point = int(len(hcs_sun_model) * 0.8)  # 80% for training, 20% for testing

#Selecting filters
features = hcs_sun_model[['monthly_sunspots','year','month']]

#Scaling features
scaler = StandardScaler()

# Fit the scaler to your features and transform them
scaled = scaler.fit_transform(features)

#Putting them in a df for the patition
X = pd.DataFrame(scaled, columns=features.columns, index=features.index)

#Outcome variables
y_std = hcs_sun_model['SD_70']
y_slope = hcs_sun_model['SL_70_log10']


In [9]:
# # Training and testing split for standard deviation prediction
X_train_std = X.iloc[:split_point]
X_test_std = X.iloc[split_point:]
y_train_std = y_std.iloc[:split_point]
y_test_std = y_std.iloc[split_point:]

# # Training and testing split for slope prediction
X_train_slope = X.iloc[:split_point]
X_test_slope = X.iloc[split_point:]
y_train_slope = y_slope.iloc[:split_point]
y_test_slope = y_slope.iloc[split_point:]

### Linear regression

In [10]:
# # Model for predicting standard deviation
model_std = LinearRegression().fit(X_train_std, y_train_std)

# # Model for predicting slope
model_slope = LinearRegression().fit(X_train_slope, y_train_slope)

# # Evaluating the models using R^2 score or other relevant metrics
std_score = model_std.score(X_test_std, y_test_std)


slope_score = model_slope.score(X_test_slope, y_test_slope)

print("STD Model R^2:", std_score)
print("Slope Model R^2:", slope_score)

STD Model R^2: 0.5379844497275629
Slope Model R^2: 0.5387953928646008


### Decision Tree

In [11]:
# For standard deviation prediction
dt_model_std = DecisionTreeRegressor().fit(X_train_std, y_train_std)
dt_std_score = dt_model_std.score(X_test_std, y_test_std)

# For slope prediction
dt_model_slope = DecisionTreeRegressor().fit(X_train_slope, y_train_slope)
dt_slope_score = dt_model_slope.score(X_test_slope, y_test_slope)

print("Decision Tree STD Model R^2:", dt_std_score)
print("Decision Tree Slope Model R^2:", dt_slope_score)


Decision Tree STD Model R^2: 0.19492340514740136
Decision Tree Slope Model R^2: 0.27049762451332604


### Random Forest

In [12]:
from sklearn.ensemble import RandomForestRegressor

# For standard deviation prediction
rf_model_std = RandomForestRegressor().fit(X_train_std, y_train_std)
rf_std_score = rf_model_std.score(X_test_std, y_test_std)

# For slope prediction
rf_model_slope = RandomForestRegressor().fit(X_train_slope, y_train_slope)
rf_slope_score = rf_model_slope.score(X_test_slope, y_test_slope)

print("Random Forest STD Model R^2:", rf_std_score)
print("Random Forest Slope Model R^2:", rf_slope_score)


Random Forest STD Model R^2: 0.3556847054320107
Random Forest Slope Model R^2: 0.237658245687545


### Gradient Boost

In [13]:
# For standard deviation prediction
gb_model_std = GradientBoostingRegressor().fit(X_train_std, y_train_std)
gb_std_score = gb_model_std.score(X_test_std, y_test_std)

# For slope prediction
gb_model_slope = GradientBoostingRegressor().fit(X_train_slope, y_train_slope)
gb_slope_score = gb_model_slope.score(X_test_slope, y_test_slope)

print("Gradient Boosting STD Model R^2:", gb_std_score)
print("Gradient Boosting Slope Model R^2:", gb_slope_score)


Gradient Boosting STD Model R^2: 0.39568187586716197
Gradient Boosting Slope Model R^2: 0.23049367430470613


In [14]:
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [15]:
# Lasso Regression
lasso_std = Lasso(alpha=0.1).fit(X_train_std, y_train_std)
lasso_slope = Lasso(alpha=0.1).fit(X_train_slope, y_train_slope)
print("Lasso STD R^2:", lasso_std.score(X_test_std, y_test_std))
print("Lasso Slope R^2:", lasso_slope.score(X_test_slope, y_test_slope))


Lasso STD R^2: 0.4919433925742881
Lasso Slope R^2: 0.16709840060981218


In [16]:
# Ridge Regression
ridge_std = Ridge(alpha=0.1).fit(X_train_std, y_train_std)
ridge_slope = Ridge(alpha=0.1).fit(X_train_slope, y_train_slope)
print("Ridge STD R^2:", ridge_std.score(X_test_std, y_test_std))
print("Ridge Slope R^2:", ridge_slope.score(X_test_slope, y_test_slope))


Ridge STD R^2: 0.5378143329290461
Ridge Slope R^2: 0.5386447104507741


In [17]:
# Polynomial Regression
poly_std = make_pipeline(PolynomialFeatures(degree=2), LinearRegression()).fit(X_train_std, y_train_std)
poly_slope = make_pipeline(PolynomialFeatures(degree=2), LinearRegression()).fit(X_train_slope, y_train_slope)
print("Polynomial STD R^2:", poly_std.score(X_test_std, y_test_std))
print("Polynomial Slope R^2:", poly_slope.score(X_test_slope, y_test_slope))



Polynomial STD R^2: 0.528332599789744
Polynomial Slope R^2: 0.3903875124881989


In [18]:
# Ensemble Model (simple averaging for demonstration)
predictions_lasso_std = lasso_std.predict(X_test_std)
predictions_ridge_std = ridge_std.predict(X_test_std)
predictions_poly_std = poly_std.predict(X_test_std)

avg_predictions_std = (predictions_lasso_std + predictions_ridge_std + predictions_poly_std) / 3
ensemble_std_score = r2_score(y_test_std, avg_predictions_std)
print("Ensemble STD Model R^2:", ensemble_std_score)

# Repeat for slope, if desired
predictions_lasso_slope = lasso_slope.predict(X_test_slope)
predictions_ridge_slope = ridge_slope.predict(X_test_slope)
predictions_poly_slope = poly_slope.predict(X_test_slope)

avg_predictions_slope = (predictions_lasso_slope + predictions_ridge_slope + predictions_poly_slope) / 3
ensemble_slope_score = r2_score(y_test_slope, avg_predictions_slope)
print("Ensemble Slope Model R^2:", ensemble_slope_score)

Ensemble STD Model R^2: 0.6748495264380412
Ensemble Slope Model R^2: 0.5805798079902817


In [19]:
from sklearn.ensemble import ExtraTreesRegressor

# For the standard deviation prediction
et_model_std = ExtraTreesRegressor(n_estimators=100, random_state=42).fit(X_train_std, y_train_std)
et_std_score = et_model_std.score(X_test_std, y_test_std)
print("Extra Trees STD Model R^2:", et_std_score)

# For the slope prediction
et_model_slope = ExtraTreesRegressor(n_estimators=100, random_state=42).fit(X_train_slope, y_train_slope)
et_slope_score = et_model_slope.score(X_test_slope, y_test_slope)
print("Extra Trees Slope Model R^2:", et_slope_score)


Extra Trees STD Model R^2: 0.3689940332697761
Extra Trees Slope Model R^2: 0.3074072503848383


### Cross Validation using a Time Series Split

In [20]:
tscv = TimeSeriesSplit(n_splits=5)  # Adjust the number of splits based on your dataset

# Initialize your model - you can use the same model or different models for std and slope
model_std = LinearRegression()
model_slope = LinearRegression()

# Evaluate the model for standard deviation
scores_std = cross_val_score(model_std, X, y_std, cv=tscv, scoring='r2')
print("R^2 scores for STD across time splits:", scores_std)

# Evaluate the model for slope
scores_slope = cross_val_score(model_slope, X, y_slope, cv=tscv, scoring='r2')
print("R^2 scores for Slope across time splits:", scores_slope)

R^2 scores for STD across time splits: [ 0.18116071  0.70232017  0.62678013 -0.29133909  0.55595975]
R^2 scores for Slope across time splits: [ 0.11237339  0.3223393   0.6285844  -0.66030711  0.46204864]
